# Aggregation Functions

Aggregation functions are mathematical constructs used to combine multiple input values into a single representative output. In the context of multi-criteria decision-making and optimization, these functions are often used to aggregate the results of desirability functions applied to various properties of an object.

These functions depend on multiple variables and parameters, which can be categorized into three main types:

1. **Input Variables**: These are the values to be aggregated, typically the outputs of desirability functions, represented as $y = (y_1, y_2, ..., y_n)$, where each $y_i \in [0, 1]$.

2. **Weights**: These are the importance factors assigned to each input variable, denoted as $w = (w_1, w_2, ..., w_n)$, where typically $\sum_{i=1}^n w_i = 1$ and $w_i \geq 0$ for all $i$.

3. **Shape Parameters**: These define the specific form of the aggregation function, denoted as $\theta = (\theta_1, \theta_2, ..., \theta_m)$, where each $\theta_j$ has its specific type and valid range.

The general form of an aggregation function can be expressed as:

$A(y, w; \theta) : [0, 1]^n \times \mathbb{R}^n_+ \times \Theta \rightarrow [0, 1]$

where $\Theta$ represents the parameter space.

## Mathematical Formulation

Let $G$ be the set of all aggregation functions in our family. For each function $A \in G$, we define:

$\text{parameters}(A) = \{\theta_1, \theta_2, ..., \theta_m\}$

Where each parameter $\theta_j$ is defined by a tuple:

$\theta_j = (\text{name}, \text{type}, \text{default}, \text{range}, \text{description})$

The validation process for a given input parameter set $\theta$ can be represented as:

$\text{validate}(A, \theta) = \begin{cases} 
      \text{True} & \text{if } \forall \theta_j \in \text{parameters}(A), \theta_j \in \text{range}(\theta_j) \\
      \text{False} & \text{otherwise}
   \end{cases}$

Where $\text{range}(\theta_j)$ represents the valid range for parameter $\theta_j$.

The computation of the aggregation function can then be expressed as:

$A(y, w; \theta) : [0, 1]^n \times \mathbb{R}^n_+ \times \Theta \rightarrow [0, 1]$

Where $y = (y_1, y_2, ..., y_n)$ represents the input variables (typically desirability function outputs), $w = (w_1, w_2, ..., w_n)$ represents the weights, and $\Theta$ is the parameter space defined by the validated parameters.

## API Description

The implementation of aggregation functions follows a similar pattern to desirability functions:

1. **Initialization**: The strategy is initialized with the shape parameters $\theta$.

   ```python
   aggregation_instance = aggregation_class(params=shape_parameters)
   ```

2. **Computation**: The aggregation function is computed for given inputs $y$ and weights $w$.

   ```python
   result = aggregation_instance(y=desirability_outputs, w=weights)
   ```

This implementation allows for flexible use of different aggregation strategies while maintaining a consistent interface.

## Implementation Details

Like the desirability functions, each aggregation function in our framework:

1. Exposes a detailed description of its required parameters
2. Embeds reasonable default values where possible
3. Defines acceptable ranges for parameters when appropriate

The input parameter values are validated against this internal definition, raising descriptive error messages when requirements are not met.

Common examples of aggregation functions include:

1. **Weighted Arithmetic Mean**: $A(y, w) = \sum_{i=1}^n w_i y_i$
2. **Weighted Geometric Mean**: $A(y, w) = \prod_{i=1}^n y_i^{w_i}$


Each of these functions may have additional shape parameters that modify their behavior, such as a power parameter for generalized means.

This framework of aggregation functions complements the desirability functions, allowing for sophisticated multi-criteria decision-making and optimization strategies.





## Code Examples

In [ ]:
# import the library and print the version
import pumas
print(pumas.__version__)

### Discover available aggregation functions
A catalogue contains all the aggregation functions implemented in PUMAS.   
The catalogue can be extended by registering into it new classes that adhere to the Aggregation interface. 

In [1]:
from pumas.aggregation import aggregation_catalogue
aggregation_catalogue.list_items()

['arithmetic_mean',
 'geometric_mean',
 'harmonic_mean',
 'deviation_index',
 'summation',
 'product']

The catalogue yields an aggregation function class, not an instance.


In [2]:
aggregation_class = aggregation_catalogue.get("geometric_mean")

## Initialize and use an aggregation function
The instantiation in most cases does not require any parameter. The only exception is the deviation_index.


In [4]:
aggregation_instance = aggregation_class()

Once properly instantiated, the aggregation function can be used to calculate an aggregated score from multiple desirability scores. 

In [7]:
values= [0.1, 0.5, 0.6]
weights = [3.0, 5.0, 2.0]
result = aggregation_instance(values=values, weights=weights)
print(f"The aggregated score is {result:.2f}")

The aggregated score is 0.32


Weights are normalized as part of the computation: their relative value matters.

In [8]:
values= [0.1, 0.5, 0.6]
weights = [0.3, 0.5,0.2]
result = aggregation_instance(values=values, weights=weights)
print(f"The aggregated score is {result:.2f}")

The aggregated score is 0.32


If no weights are provided, equal weights are assumed

In [11]:
values= [0.1, 0.5, 0.6]
weights = [1.0, 1.0, 1.0]
result = aggregation_instance(values=values, weights=weights)
print(f"The aggregated score is {result:.2f}")

The aggregated score is 0.36


In [12]:
values= [0.1, 0.5, 0.6]
result = aggregation_instance(values=values, )
print(f"The aggregated score is {result:.2f}")

The aggregated score is 0.36


if either a value or a weight is None or Nan, the pair is deleted, and a warning is raised. 

In [18]:
vw = [
    ([0.1, None, 0.6], [0.3, 0.5, 0.2]),
    ([0.1, None, 0.6], [0.3,None, 0.2]),
    ([0.1, 0.5, 0.6], [0.3, None, 0.2]),
    ([0.1, 0.6], [0.3, 0.2]),
 ]

for (values, weights) in vw:
    result = aggregation_instance(values=values, weights=weights)
    print(result)

0.21897503240933447
0.21897503240933447
0.21897503240933447
0.21897503240933447


### Discover Parameters
Most aggregation function do not require parameters, while some do.

In [9]:
aggregation_class = aggregation_catalogue.get("geometric_mean")
aggregation_instance = aggregation_class()
print(aggregation_instance.parameters_map)

{}


In [10]:
aggregation_class = aggregation_catalogue.get("deviation_index")
aggregation_instance = aggregation_class()
print(aggregation_instance.parameters_map)

{'ideal_value': FloatParameter(name='ideal_value', default=1.0, min=-inf, max=inf)}


### Errors while providing input

In [15]:
values= 1
weights = [1.0, 1.0,1.0]
try:
    result = aggregation_instance(values=values, weights=weights)
except Exception as e:
    print(e)

object of type 'int' has no len()
